# Day 5: Hydrothermal Venture

## Part 1

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```

In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

Your puzzle answer was `7473`.

In [1]:
with open('input/day5/input.txt', 'r') as finput:
    raw_input = [x.strip() for x in finput.readlines()]

In [2]:
class Point:
    def __init__(self, x, y):
        self.x = int(x)
        self.y = int(y)

    def __repr__(self):
        return f"({self.x},{self.y})"

    def __sub__(self, other):
        from math import sqrt
        return sqrt((other.x - self.x)**2 + (other.y - self.y)**2)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

class Line:
    def __init__(self, x1, y1, x2, y2):
        self.start = Point(x1, y1)
        self.end = Point(x2, y2)

    @classmethod
    def from_string(cls, point_map: str):
        start, end = point_map.split(" -> ")
        x1, y1 = start.split(",")
        x2, y2 = end.split(",")
        return cls(x1, y1, x2, y2)

    @property
    def length(self):
        return self.start - self.end

    @property
    def horizontal(self):
        return self.start.y == self.end.y

    @property
    def vertical(self):
        return self.start.x == self.end.x

    @property
    def v_or_h(self):
        return self.horizontal or self.vertical

    def get_all_points(self):
        if self.horizontal:
            incr = 1 if self.start.x - self.end.x > 0 else -1
            for x in range(self.end.x, self.start.x + incr, incr):
                yield Point(x, self.start.y)
        elif self.vertical:
            incr = 1 if self.start.y - self.end.y > 0 else -1
            for y in range(self.end.y, self.start.y + incr, incr):
                yield Point(self.start.x, y)
        else:
            raise NotImplementedError("Only vertical or horizontal lines support getting points")


    def __repr__(self):
        return f"Length: {self.length} VorH: {self.v_or_h} From: {self.start} To: {self.end}"

class Map:

    MAX_SIZE = 1000

    def __init__(self, line_class=Line, point_class=Point):
        self.coords = [[0 for x in range(self.MAX_SIZE)] for x in range(self.MAX_SIZE)]
        self.lines = list()
        self.hotspots = 0

        self.line_class = line_class
        self.point_class = point_class

    def load_line(self, line: str):
        self.lines.append(self.line_class.from_string(line))

    def load_lines(self, lines: [str]):
        for line in lines:
            self.load_line(line)

    def map_lines(self):
        for line in self.lines:
            try:
                for point in line.get_all_points():
                    if self.coords[point.x][point.y] == 1:
                        self.hotspots += 1

                    self.coords[point.x][point.y] += 1
            except NotImplementedError:
                # we're okay with this, not all lines are supported always
                continue

In [3]:
vent_map = Map()
vent_map.load_lines(raw_input)

In [4]:
vent_map.map_lines()

In [5]:
print(f"There are {vent_map.hotspots} found hotspots.")

There are 7473 found hotspots.


## Part Two

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

    An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
    An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:

```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

Your puzzle answer was `24164`.

In [6]:
class LineP2(Line):

    @property
    def diagonal(self):
        return self.slope == 1 or self.slope == -1

    @property
    def slope(self):
        ydiff = (self.end.y - self.start.y)
        xdiff = (self.end.x - self.start.x)
        return ydiff / xdiff if xdiff else None

    @property
    def y_intercept(self):
        return self.start.y - self.slope * self.start.x

    def get_all_points(self):
        if self.diagonal:
            xincr = 1 if self.end.x - self.start.x > 0 else -1
            yincr = 1 if self.end.y - self.start.y > 0 else -1

            cur = self.start
            yield cur  # we'll be modifying it immediately in the loop, so yield first

            while cur != self.end:
                cur = Point(cur.x + xincr, cur.y + yincr)
                yield cur
        elif self.v_or_h:
            yield from super().get_all_points()
        else:
            raise NotImplementedError("Only vertical, horizontal, diagonal lines support getting points")

    def __repr__(self):
        return f"Length: {self.length} Slope: {self.slope} From: {self.start} To: {self.end}"

In [7]:
vent_map_p2 = Map(line_class=LineP2)
vent_map_p2.load_lines(raw_input)

In [8]:
vent_map_p2.map_lines()

In [9]:
print(f"There are {vent_map_p2.hotspots} found hotspots.")

There are 24164 found hotspots.
